# 🎨 DreamScape AI — Multimodal Generation Notebook

This notebook generates **dream-inspired multimodal outputs** — combining *text, audio, and visuals* using large generative models.  
It integrates:
- Stable Diffusion (SD-Turbo) for **illustrations**
- Faster-Whisper for **speech transcription**
- MusicGen (via 🤗 Transformers) for **ambient sound**
- Detoxify for **safety filtering**
- NetworkX for **motif graphing**

All assets and metadata are saved under `results/` for easy review.

In [1]:
# --- Environment Setup ---
%pip -q install --upgrade diffusers accelerate pillow gradio soundfile librosa faster-whisper detoxify torch torchvision torchaudio transformers networkx matplotlib scipy

Note: you may need to restart the kernel to use updated packages.


##  Imports and Paths

In [2]:
import os, gc, io, math, json, random, warnings, tempfile, re, itertools
import numpy as np
import soundfile as sf
import librosa
from PIL import Image, ImageDraw, ImageFont, ImageFilter
from pathlib import Path
from datetime import datetime
import torch
import matplotlib.pyplot as plt
import networkx as nx
from detoxify import Detoxify
from transformers import pipeline
import gradio as gr

warnings.filterwarnings("ignore")

OUT_DIR = Path("results")
OUT_DIR.mkdir(exist_ok=True)

##  Load Stable Diffusion Turbo

In [3]:
from diffusers import AutoPipelineForText2Image

SafetyChecker = None
ClipProcessor = None
try:
    from diffusers.pipelines.stable_diffusion import StableDiffusionSafetyChecker as SafetyChecker
except Exception:
    try:
        from diffusers.pipelines.stable_diffusion.safety_checker import StableDiffusionSafetyChecker as SafetyChecker
    except Exception:
        SafetyChecker = None

try:
    from transformers import CLIPImageProcessor as ClipProcessor
except Exception:
    try:
        from transformers import AutoImageProcessor as ClipProcessor
    except Exception:
        ClipProcessor = None

TXT2IMG_ID = "stabilityai/sd-turbo"

device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)
dtype = torch.float16 if device == "cuda" else torch.float32

safety_checker = None
feature_extractor = None
if SafetyChecker and ClipProcessor:
    try:
        safety_checker = SafetyChecker.from_pretrained("CompVis/stable-diffusion-safety-checker")
        feature_extractor = ClipProcessor.from_pretrained("openai/clip-vit-base-patch32")
    except Exception as e:
        warnings.warn(f"Safety checker unavailable: {e}")

pipe_kwargs = dict(dtype=dtype)
if safety_checker and feature_extractor:
    pipe_kwargs["safety_checker"] = safety_checker
    pipe_kwargs["feature_extractor"] = feature_extractor

TXT2IMG_PIPE = AutoPipelineForText2Image.from_pretrained(TXT2IMG_ID, **pipe_kwargs).to(device)
try:
    TXT2IMG_PIPE.enable_attention_slicing()
except Exception:
    pass

print(f"Loaded text→image: {TXT2IMG_ID} on {device} (dtype={dtype}); "
      f"safety={'ON' if safety_checker else 'OFF (blur fallback)'}")

Keyword arguments {'dtype': torch.float32} are not expected by StableDiffusionPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Loaded text→image: stabilityai/sd-turbo on mps (dtype=torch.float32); safety=ON


## Helper Functions

In [4]:
def set_seed(seed: int | None):
    if seed is None:
        return
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def sanitize_prompt(prompt: str):
    """Filter toxic or unsafe language before generation."""
    try:
        scores = Detoxify('original').predict(prompt)
        tox_score = float(scores.get('toxicity', 0.0))
    except Exception:
        tox_score = 0.0
    cleaned = prompt.strip()
    softened = False
    if tox_score > 0.5:
        cleaned = "A calm, imaginative reinterpretation of " + prompt
        softened = True
    return cleaned, tox_score, softened

def save_image(img: Image.Image, path: Path) -> Path:
    img.save(path, format="PNG")
    return path

def now_stamp():
    return datetime.now().strftime("%Y%m%d_%H%M%S")

## Audio Generation (MusicGen via 🤗 Transformers)

In [5]:
# --- Step 4: Audio Generation (MusicGen via 🤗 Transformers, or Ambient Fallback) ---
import numpy as np
import scipy.io.wavfile as wavfile
from pathlib import Path

from transformers import pipeline, AutoProcessor, MusicgenForConditionalGeneration

MUSICGEN_MODEL_ID = "facebook/musicgen-small"   # you can try "facebook/musicgen-medium" later
# Use CPU for maximum compatibility (CUDA if you have NVIDIA, but CPU is fine here)
TTA_DEVICE = -1

TTA_PIPE = None
TTA_DIRECT = None
AUDIO_ENGINE = "AmbientFallback"

def _print_versions():
    import transformers, torch, scipy, huggingface_hub
    print(
        "versions =>",
        "transformers", transformers.__version__,
        "| torch", torch.__version__,
        "| scipy", scipy.__version__,
        "| hf-hub", huggingface_hub.__version__,
    )

_print_versions()

# 1) Try the Transformers pipeline("text-to-audio") first
try:
    TTA_PIPE = pipeline(
        task="text-to-audio",
        model=MUSICGEN_MODEL_ID,
        device=TTA_DEVICE,  # -1=CPU
    )
    AUDIO_ENGINE = f"MusicGen/Transformers({MUSICGEN_MODEL_ID})"
    print(f"🎵 Loaded MusicGen via pipeline: {MUSICGEN_MODEL_ID}")
except Exception as e:
    print("⚠️ pipeline(text-to-audio) failed:", e)
    TTA_PIPE = None

# 2) If pipeline failed, try direct model+processor
if TTA_PIPE is None:
    try:
        proc = AutoProcessor.from_pretrained(MUSICGEN_MODEL_ID)
        mdl  = MusicgenForConditionalGeneration.from_pretrained(MUSICGEN_MODEL_ID)
        mdl = mdl.to("cpu")
        TTA_DIRECT = (proc, mdl)
        AUDIO_ENGINE = f"MusicGen/Direct({MUSICGEN_MODEL_ID})"
        print(f"🎵 Loaded MusicGen via direct model: {MUSICGEN_MODEL_ID}")
    except Exception as e:
        print("⚠️ Direct MusicGen load failed:", e)
        TTA_DIRECT = None
        AUDIO_ENGINE = "AmbientFallback"
        print("➡️ Will use ambient fallback for audio.")

def text_to_audio(prompt: str, path: Path, seconds: int = 8) -> Path:
    """
    Prefer MusicGen (pipeline). If unavailable, try direct model.
    If both fail, write a quiet sine tone (ambient fallback).
    """
    path.parent.mkdir(parents=True, exist_ok=True)

    # A) Pipeline route
    if TTA_PIPE is not None:
        try:
            # ~50 tokens/sec is a good rough target
            result = TTA_PIPE(
                prompt,
                forward_params={"do_sample": True, "max_new_tokens": int(seconds * 50)}
            )
            sr = int(result["sampling_rate"])
            audio = result["audio"]  # float32 mono [-1,1]
            wavfile.write(path, rate=sr, data=(audio * 32767).astype(np.int16))
            return path
        except Exception as e:
            print("⚠️ MusicGen pipeline failed, trying direct:", e)

    # B) Direct route
    if TTA_DIRECT is not None:
        try:
            proc, mdl = TTA_DIRECT
            inputs = proc(text=[prompt], padding=True, return_tensors="pt")
            with torch.no_grad():
                # seconds * 50 tokens ≈ rough duration control
                audio_values = mdl.generate(**inputs, max_new_tokens=int(seconds * 50))
            sr = mdl.config.audio_encoder.sampling_rate
            audio = audio_values[0, 0].cpu().numpy()  # (samples,)
            wavfile.write(path, rate=sr, data=(audio * 32767).astype(np.int16))
            return path
        except Exception as e:
            print("⚠️ MusicGen direct failed, falling back:", e)

    # C) Fallback (quiet sine)
    sr = 22050
    t = np.linspace(0, 5, int(5 * sr), endpoint=False)
    tone = 0.02 * np.sin(2 * np.pi * 220 * t)
    wavfile.write(path, sr, (tone * 32767).astype(np.int16))
    return path

print(f"🔊 AUDIO_ENGINE active: {AUDIO_ENGINE}")

versions => transformers 4.57.1 | torch 2.9.0 | scipy 1.16.3 | hf-hub 0.36.0


Device set to use cpu


🎵 Loaded MusicGen via pipeline: facebook/musicgen-small
🔊 AUDIO_ENGINE active: MusicGen/Transformers(facebook/musicgen-small)


## Text → Image and Multimodal Generation

In [6]:
def text_to_image(
    prompt: str,
    negative: str = "low quality, blurry, watermark, text, logo",
    steps: int = 4,
    guidance: float = 0.0,
    height: int = 512,
    width: int = 512,
    seed: int | None = 1234
) -> Image.Image:
    set_seed(seed)
    try:
        out = TXT2IMG_PIPE(
            prompt=prompt,
            negative_prompt=negative,
            num_inference_steps=steps,
            guidance_scale=guidance,
            height=height, width=width,
        )
        img = out.images[0]
        flagged = False
        if hasattr(out, "nsfw_content_detected") and out.nsfw_content_detected:
            flagged = bool(out.nsfw_content_detected[0])
        if flagged:
            img = img.filter(ImageFilter.GaussianBlur(radius=24))
        return img
    except Exception as e:
        print("⚠️ Generation retry on CPU:", e)
        pipe_cpu = TXT2IMG_PIPE.to("cpu")
        out = pipe_cpu(
            prompt=prompt,
            negative_prompt=negative,
            num_inference_steps=max(steps, 8),
            guidance_scale=max(guidance, 1.0),
            height=height, width=width,
        )
        img = out.images[0]
        return img

 ## Moodboard + Motif Graph

In [7]:
def _square_thumb(im: Image.Image, size=384):
    w, h = im.size
    s = min(w, h)
    im = im.crop(((w-s)//2, (h-s)//2, (w+s)//2, (h+s)//2))
    return im.resize((size, size), Image.LANCZOS)

def compose_grid(images, cols=3, pad=8, bg=(18,18,18)):
    tiles = [_square_thumb(img) for img in images]
    w, h = tiles[0].size
    rows = math.ceil(len(tiles)/cols)
    W, H = cols*w+(cols+1)*pad, rows*h+(rows+1)*pad
    canvas = Image.new("RGB", (W,H), bg)
    for i, t in enumerate(tiles):
        r, c = divmod(i, cols)
        canvas.paste(t, (pad+c*(w+pad), pad+r*(h+pad)))
    return canvas

def text_to_moodboard(text: str, n_images=6, cols=3):
    base_prompt, _, _ = sanitize_prompt(text)
    styles = [
        "surreal cinematic volumetric light", "dreamlike watercolor pastel",
        "digital art neon synthwave", "oil painting baroque lighting",
        "minimalist muted palette", "storybook ink & wash"
    ]
    imgs=[]
    for i in range(n_images):
        styl = styles[i % len(styles)]
        prompt_i = f"{base_prompt}. Style: {styl}"
        img = text_to_image(prompt_i, seed=1234+i)
        imgs.append(img)
    return compose_grid(imgs, cols=cols)

In [8]:
# --- Smarter Motif Graph (NER-backed with lexicon fallback) ---
import re
import networkx as nx
import matplotlib.pyplot as plt

# Try to load a lightweight NER pipeline
NER_PIPE = None
try:
    from transformers import pipeline as hf_pipeline
    # CPU is fine; this model is small and downloads once
    NER_PIPE = hf_pipeline(
        task="token-classification",
        model="dslim/bert-base-NER",
        aggregation_strategy="simple",
        device=-1
    )
    print("🧩 Motif NER: dslim/bert-base-NER loaded.")
except Exception as e:
    print("⚠️ Motif NER unavailable, using lexicon only.", e)
    NER_PIPE = None

# Expanded motif lexicon (common dream nouns & symbols)
MOTIF_LEXICON = {
    "forest","tree","river","sea","ocean","wave","water","rain","storm","cloud","sky","moon","sun","star","night","light","shadow",
    "mirror","glass","window","door","stairs","bridge","room","house","city","desert","mountain","valley","garden","flower",
    "bird","cat","dog","fish","horse","person","child","friend","stranger",
    "clock","time","book","key","phone","car","train","boat","plane",
    "color","red","blue","green","gold","violet","purple","black","white",
    "fire","ice","snow","fog","mist","sand"
}

_token = re.compile(r"[A-Za-z']+")

def _heuristic_motifs(text: str):
    """Fallback: pick words that intersect our lexicon."""
    toks = [w.lower() for w in _token.findall(text)]
    return [w for w in toks if w in MOTIF_LEXICON]

def _ner_motifs(text: str):
    """Use NER to extract salient tokens (LOC/ORG/PER/MISC labels)."""
    if NER_PIPE is None:
        return []
    ents = NER_PIPE(text)
    # Keep reasonably short entity strings; split multiword into tokens as motifs too
    motifs = []
    for e in ents:
        span = e.get("word", "") or e.get("entity_group", "")
        span = span.strip()
        if not span:
            continue
        # break multiword spans into individual tokens plus the span itself
        parts = [w.lower() for w in _token.findall(span)]
        if span and len(span) <= 24:
            motifs.append(span.lower())
        motifs.extend(parts)
    # If NER returned nothing, return empty (caller will fallback/merge)
    return [m for m in motifs if len(m) >= 2]

def extract_motifs(text: str, top_k: int = 20):
    """Combine NER + lexicon, deduplicate, keep order of appearance."""
    ner = _ner_motifs(text)
    lex = _heuristic_motifs(text)
    combined = []
    seen = set()
    for w in ner + lex:
        if w not in seen:
            combined.append(w)
            seen.add(w)
        if len(combined) >= top_k:
            break
    return combined

def build_motif_graph(text: str, window: int = 4, min_weight: int = 1):
    """
    Build a co-occurrence graph over extracted motifs within a sliding window.
    Keeps even single co-occurrences by default to avoid an empty graph.
    """
    motifs = extract_motifs(text, top_k=40)
    G = nx.Graph()
    # Add nodes even if we end with no edges—so the graph isn’t blank.
    for m in motifs:
        G.add_node(m)

    # Co-occurrence edges
    for i in range(len(motifs)):
        for j in range(i + 1, min(i + window, len(motifs))):
            if motifs[i] != motifs[j]:
                a, b = sorted((motifs[i], motifs[j]))
                w = G.get_edge_data(a, b, default={'weight': 0})['weight'] + 1
                G.add_edge(a, b, weight=w)

    # Prune very weak edges if desired
    if min_weight > 1:
        G.remove_edges_from([(u, v) for u, v, d in G.edges(data=True) if d['weight'] < min_weight])

    return G

def draw_motif_graph(G: nx.Graph, out_path=None):
    """
    Draws the motif graph. If there are nodes but no edges, shows isolated nodes with a note.
    """
    fig, ax = plt.subplots(figsize=(5.2, 5.2))
    if G.number_of_nodes() == 0:
        ax.text(0.5, 0.5, "No motifs found", ha="center", va="center")
        ax.axis("off")
    else:
        if G.number_of_edges() == 0:
            # Place nodes in a circle to avoid overlap, add a gentle note
            pos = nx.circular_layout(G)
            nx.draw_networkx_nodes(G, pos, node_color="#91b4ff", node_size=900, ax=ax)
            nx.draw_networkx_labels(G, pos, font_size=9, ax=ax)
            ax.set_title("Motifs (no strong co-occurrence found)")
            ax.axis("off")
        else:
            pos = nx.spring_layout(G, seed=42)
            widths = [1 + d['weight'] for *_u, _v, d in G.edges(data=True)]
            nx.draw(
                G, pos,
                with_labels=True,
                node_color="#91b4ff",
                node_size=900,
                width=widths,
                edge_color="#8aa0d6",
                ax=ax
            )
            ax.axis("off")
    if out_path:
        fig.savefig(out_path, bbox_inches="tight", dpi=150)
    plt.close(fig)
    return out_path

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


🧩 Motif NER: dslim/bert-base-NER loaded.


##  Full Multimodal + Extras

In [9]:
def multimodal_from_text(user_text: str, project_tag: str = "dream", fast: bool = False):
    clean_prompt, tox_score, softened = sanitize_prompt(user_text)
    stamp = now_stamp()
    base = OUT_DIR / f"{project_tag}_{stamp}"
    base.parent.mkdir(parents=True, exist_ok=True)

    art_prompt = (
        "Dreamlike surreal illustration, cinematic light, ethereal mood, 35mm film look. "
        f"Abstract symbols: {clean_prompt[:500]}"
    )
    img = text_to_image(
        art_prompt,
        steps=3 if fast else 4,
        height=448 if fast else 512,
        width=448 if fast else 512,
    )
    img_path = save_image(img, base.with_suffix(".png"))

    audio_prompt = f"Ambient calm pads, shimmering tones, inspired by: {clean_prompt[:150]}"
    wav_path = text_to_audio(audio_prompt, base.with_suffix(".wav"))

    report = {
        "timestamp": stamp,
        "input_text": user_text,
        "used_prompt": clean_prompt,
        "toxicity_score": tox_score,
        "softened_prompt": bool(softened),
        "image_model": TXT2IMG_ID,
        "audio_engine": AUDIO_ENGINE,
        "paths": {"image": str(img_path), "audio": str(wav_path)}
    }
    with open(base.with_suffix(".json"), "w") as f:
        json.dump(report, f, indent=2)
    return report

def multimodal_with_extras(user_text: str, project_tag="dream", mood=True, motifs=True):
    report = multimodal_from_text(user_text, project_tag)
    mb_path = None
    g_path = None

    if mood:
        mb = text_to_moodboard(user_text)
        mb_path = OUT_DIR / f"{project_tag}_{now_stamp()}_moodboard.png"
        mb.save(mb_path)

    if motifs:
        G = build_motif_graph(user_text)
        g_path = OUT_DIR / f"{project_tag}_{now_stamp()}_motifs.png"
        draw_motif_graph(G, out_path=g_path)

    report["extras"] = {
        "moodboard": str(mb_path) if mb_path else None,
        "motif_graph": str(g_path) if g_path else None
    }

    # Persist the enriched report next to the image
    json_path = Path(report["paths"]["image"]).with_suffix(".json")
    with open(json_path, "w") as f:
        json.dump(report, f, indent=2)

    return report

## 🎙️ Optional: Speech-to-Text with Faster-Whisper

This section adds microphone/upload audio input. We transcribe speech to text using **Faster-Whisper** on CPU for maximum compatibility (avoids the `unsupported device mps` error on Mac).  

- Model size: `small` (you can try `tiny` for faster but less accurate).
- Compute type: `int8` for speed.
- Voice activity detection (VAD) enabled to trim silences.

In [10]:
# --- Faster-Whisper (ASR) ---
import faster_whisper
from typing import Optional

# Use CPU to avoid "unsupported device mps" issues on Mac
WHISPER_DEVICE = "cpu"           # or "cuda" if you have a working NVIDIA setup
WHISPER_MODEL_SIZE = "small"     # try "tiny" for even faster, "base"/"medium" for more accuracy
WHISPER_COMPUTE = "int8"         # good speed/accuracy tradeoff on CPU

# Create one global model instance to avoid reloading on each click
try:
    ASR_MODEL = faster_whisper.WhisperModel(
        WHISPER_MODEL_SIZE,
        device=WHISPER_DEVICE,
        compute_type=WHISPER_COMPUTE
    )
    print(f"🎤 Loaded Faster-Whisper: {WHISPER_MODEL_SIZE} on {WHISPER_DEVICE} ({WHISPER_COMPUTE})")
except Exception as e:
    ASR_MODEL = None
    print("Faster-Whisper not available:", e)

def transcribe_audio(audio_path: Optional[str]) -> str:
    """
    Transcribe an audio file path to text using Faster-Whisper.
    Returns an empty string if ASR is unavailable or no audio given.
    """
    if not audio_path or ASR_MODEL is None:
        return ""
    try:
        segments, info = ASR_MODEL.transcribe(
            audio_path,
            vad_filter=True,
            vad_parameters={"min_silence_duration_ms": 300},
            beam_size=1,
            best_of=1,
            language=None  # auto-detect
        )
        text = " ".join(seg.text.strip() for seg in segments).strip()
        return text
    except Exception as e:
        print("ASR failed:", e)
        return ""

🎤 Loaded Faster-Whisper: small on cpu (int8)


In [11]:
def run_all_with_audio(text_input, audio_input, prefer_audio, make_mood=True, make_motif=True, fast=False):
    """
    - If 'prefer_audio' is True and a microphone/upload file is provided, we transcribe and use that.
    - Otherwise we use the text box as usual.
    """
    transcript = ""
    if prefer_audio and audio_input:
        transcript = transcribe_audio(audio_input)
        if transcript:
            text_input = transcript

    text_input = (text_input or "").strip()
    if not text_input:
        return None, None, None, None, "Please enter or record a dream.", transcript

    meta = multimodal_with_extras(text_input, mood=make_mood, motifs=make_motif)
    return (
        meta["paths"]["image"],
        meta["paths"]["audio"],
        meta["extras"]["moodboard"],
        meta["extras"]["motif_graph"],
        f"Toxicity: {meta['toxicity_score']:.2f}",
        transcript
    )

# Gradio Interface

In [ ]:
# Gradio UI with microphone/upload + transcript preview 
with gr.Blocks(title="DreamScape AI") as demo:
    gr.Markdown("## DreamScape AI — Multimodal Dream Generator")

    with gr.Row():
        with gr.Column(scale=1):
            tbox = gr.Textbox(label="Describe your dream (text input)", lines=4)
            aud = gr.Audio(
                sources=["microphone", "upload"],
                type="filepath",
                label="Optional: record/upload your dream"
            )
            prefer_audio = gr.Checkbox(
                value=True,
                label="Prefer audio input if provided (use ASR transcript)"
            )
            make_mood = gr.Checkbox(value=True, label="Generate moodboard")
            make_motif = gr.Checkbox(value=True, label="Generate motif graph")
            fast = gr.Checkbox(value=False, label="Fast mode (slightly lower quality, faster)")
            run = gr.Button("Generate", variant="primary")

        with gr.Column(scale=1):
            out_img = gr.Image(label="Generated Image")
            out_aud = gr.Audio(label="Generated Audio")
            out_mb = gr.Image(label="Moodboard")
            out_g = gr.Image(label="Motif Graph")
            out_t = gr.Textbox(label="Analysis")
            out_asr = gr.Textbox(label="Transcribed Text (if audio used)")

    run.click(
        fn=run_all_with_audio,
        inputs=[tbox, aud, prefer_audio, make_mood, make_motif, fast],
        outputs=[out_img, out_aud, out_mb, out_g, out_t, out_asr]
    )

demo.launch(debug=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]